In [1]:
try:
    __IPYTHON__
    is_notebook = True
    print('Notebook mode')
except NameError:
    is_notebook = False
    print('Script mode')

Notebook mode


# Pip Install

In [2]:
if is_notebook:
    !pip install boto3 astropy sfdmap progressbar2 GPUtil parse

# Initialization

In [3]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
import boto3
from matplotlib import pyplot as plt
import time

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'thesis2')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3-west.nrp-nautilus.io'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'thesis2')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_models_dir_path = os.path.join(s3_work_dir_path , 'models')
s3_final_table_csv_path = os.path.join(s3_data_dir_path, 'SDSS_DR16_all.csv')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig

2023-02-18 20:22:06.832040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 20:22:06.960392: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-18 20:22:07.565287: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-18 20:22:07.565362: W tensorflow/

# Load Data

In [4]:
data_path_in_bucket = 'almogh/thesis2/data/BigRF/train/spec.npy'
X = from_s3_npy(s3_client = s3_client,
                bucket_name = bucket_name,
                path_in_bucket = data_path_in_bucket)

loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/spec.npy


# Load Model

In [5]:
# Load BigRF
BigRF = from_s3_pkl(s3_client = s3_client,
                      bucket_name = bucket_name,
                      path_in_bucket = 'almogh/thesis2/models/BigRF/train/rf.pkl')

loading from uri: s3://tau-astro/almogh/thesis2/models/BigRF/train/rf.pkl


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Infer

In [6]:
from uRF_SDSS import calcDisMat
dist_mat = calcDisMat(BigRF, X)

  Applying forest to results... 


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.7s finished


  Max leaf index: 
360
  Calculating real leafs... 


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 437 out of 500 | elapsed:    3.7s remaining:    0.5s


  Calculating similarity matrix... 


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.8s finished


In [7]:
# save
to_s3_npy(dist_mat,
          s3_client = s3_client,
          bucket_name = bucket_name,
          path_in_bucket = 'almogh/thesis2/eval/inference/D_BigRF.npy')

saving to uri: s3://tau-astro/almogh/thesis2/eval/inference/D_BigRF.npy


True